In [97]:
import pandas as pd
import numpy as np
import torch

In [98]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [99]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [100]:
# Read in training data
df = pd.read_csv('../public_data/train/track_a/eng.csv')
df['text_embedding'] = df['text'].map(lambda x: embedding_model.encode(x))

In [101]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text_embedding'] , df['Anger'], test_size=0.2)

In [102]:
test_df = pd.read_csv('../public_data/dev/track_a/eng_a.csv')
test_df['text_embedding'] = test_df['text'].map(lambda x: embedding_model.encode(x))

In [103]:
from lightgbm import LGBMClassifier, LGBMRegressor

predictions = {}
predictions1 = {}

for i in ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']:
    model = LGBMClassifier(
        n_estimators=1000,
        verbose=-1
    )
    model.fit(list(df['text_embedding']), df[i])
    preds = model.predict(list(test_df['text_embedding']))
    predictions.update({i: preds})

    model1 = LGBMRegressor(
        n_estimators=1000,
        verbose=-1
    )
    model1.fit(list(df['text_embedding']), df[i])
    preds = model1.predict(list(test_df['text_embedding']))
    predictions1.update({i: preds})

In [106]:
for i in ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']:
    test_df[i] = predictions[i]

In [109]:
test_df = test_df.drop(['text', 'text_embedding'], axis = 1)

In [110]:
test_df

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,0,1,0,0,0
1,eng_dev_track_a_00002,0,1,0,0,1
2,eng_dev_track_a_00003,0,1,0,0,0
3,eng_dev_track_a_00004,0,1,1,0,0
4,eng_dev_track_a_00005,0,1,0,0,0
...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,1,0,0,0
112,eng_dev_track_a_00113,0,0,0,0,1
113,eng_dev_track_a_00114,0,1,0,0,0
114,eng_dev_track_a_00115,0,0,0,1,0


In [112]:
q.to_csv('../outputs/pred_eng_a.csv', index=False)

In [111]:
zero_rows = q[q[['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']].eq(0).all(axis=1)]
zero_rows

,id,Anger,Fear,Joy,Sadness,Surprise


In [86]:
for i in zero_rows.index:
    dd = {u: x[i] for u, x in predictions1.items()}
    x = max(dd, key=dd.get)
    test_df.loc[i, x] = 1
    # print(dd)

In [87]:
test_df

,id,text,Anger,Fear,Joy,Sadness,Surprise,text_embedding
0,eng_dev_track_a_00001,"My mouth fell open `` No, no, no... I..",0,1,0,0,0,"[-0.022840476, 0.005874898, 0.057702277, -0.02..."
1,eng_dev_track_a_00002,You can barely make out your daughter's pale f...,0,1,0,0,0,"[0.08949178, 0.04194942, 0.066804454, 0.069105..."
2,eng_dev_track_a_00003,But after blinking my eyes for a few times lep...,0,0,0,0,1,"[-0.05072529, -0.04732532, -0.023370713, 0.026..."
3,eng_dev_track_a_00004,Slowly rising to my feet I came to the conclus...,0,1,1,0,0,"[-0.016266463, 0.028907906, 0.022233931, 0.115..."
4,eng_dev_track_a_00005,I noticed this months after moving in and doin...,0,1,0,0,0,"[0.06023583, -0.060040664, 0.02365117, 0.08191..."
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,"""ARcH stop your progression.",0,1,0,0,0,"[0.038538784, 0.013606158, 0.039151467, -0.067..."
112,eng_dev_track_a_00113,"This 'star', starts to move across the sky.",0,0,0,0,1,"[-0.0057569565, -0.036201872, 0.0046417764, 0...."
113,eng_dev_track_a_00114,and my feet hurt.,0,1,0,0,0,"[-0.035371713, -0.071153864, 0.097622715, 0.06..."
114,eng_dev_track_a_00115,so i cried my eyes out and did the drawing.,0,0,0,1,0,"[0.039935336, 0.033184722, 0.023528507, -0.005..."
